# ResNet kvantizácia

## Obsah

*   [Predspracovanie](#predspracovanie)
<a href='#predspracovanie'> </a>
*   [Statistiky na staticku kvant.](#stats)
<a href='#stats'> </a>
*   [Base precision](#base)
<a href='#base'> </a>
*   Range-Based Quantization
    *   [8 bit kvantizacia](#8bit)
<a href='#8bit'> </a>
    *   [4 bit kvantizacia](#4bit)
<a href='#4bit'> </a>
*   Loss-Aware Quantization
    *   [8 bit kvantizacia](#8bit-loss)
<a href='#8bit-loss'> </a>
    *   [4 bit kvantizacia](#4bit-loss)
<a href='#4bit-loss'> </a>

In [1]:
import torch
import torchvision
import distiller
from distiller.models import create_model


print(torch.__version__)
print(torchvision.__version__)

1.3.1
0.4.2


<a id='predspracovanie'> </a>

# Predspracovanie

In [2]:
model = create_model(pretrained=True,dataset='imagenet',arch='resnet18') 

In [3]:
model

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): DistillerBasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (add): EltwiseAdd()
        (relu2): ReLU(inplace=True)
      )
      (1): DistillerBasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bia

In [4]:
import numpy as np
from torchvision import transforms
# from torch.utils.data import DataLoader

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
preprocessing = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])

torch.manual_seed(0)
np.random.seed(0)

batch_size = 40
num_workers = 1
dataset = torchvision.datasets.ImageFolder('/home/bohumil/FIIT/BP/BP/Zdroje_kod/imagenet/val'
                                           ,preprocessing)

small, big = torch.utils.data.random_split(dataset,[7000, len(dataset)-7000])
 
dataloader = torch.utils.data.DataLoader(small,batch_size=batch_size,
                                         num_workers=num_workers,shuffle=True)


In [5]:
from resnet_output import resnet_output

def target_labels(dataset,target):
    list = target.tolist()
    for i in range(len(list)):
        list[i] = dataset.classes[list[i]]
        list[i] = resnet_output[list[i]]
    return torch.LongTensor(list)

In [6]:
# z <distiller_root>/jupyter/post_train_quant_convert_pytorch.ipynb
import torchnet as tnt
import math
import numpy as np


def eval_model(data_loader, model, device='cpu', print_freq=10):
    # print('Evaluation model ', model.arch)
    
    criterion = torch.nn.CrossEntropyLoss().to(device)
    
    loss = tnt.meter.AverageValueMeter()
    classerr = tnt.meter.ClassErrorMeter(accuracy=True, topk=(1, 5))
    # apmeter = tnt.meter.APMeter()

    total_samples = len(data_loader.sampler)
    batch_size = data_loader.batch_size
    total_steps = math.ceil(total_samples / batch_size)
    print('{0} samples ({1} per mini-batch)'.format(total_samples, batch_size))

    # Switch to evaluation mode
    model.eval()

    for step, (inputs, target) in enumerate(data_loader):
        with torch.no_grad():
            inputs, target = inputs.to(device), target.to(device)
            # compute output from model
            output = model(inputs)
            target = target_labels(dataset,target).to(device)
            # compute loss and measure accuracy
            loss.add(criterion(output, target).item())
            classerr.add(output.data, target)

            if (step + 1) % print_freq == 0:
                print('[{:3d}/{:3d}] Top1: {:.3f}  Top5: {:.3f}  Loss: {:.3f}'.format(
                      step + 1, total_steps, classerr.value(1), classerr.value(5), loss.mean), flush=True)
    print('----------')
    print('Overall ==> Top1: {:.3f}  Top5: {:.3f}  Loss: {:.3f}  PPL: {:.3f}'.format(
        classerr.value(1), classerr.value(5), loss.mean, np.exp(loss.mean)), flush=True)

In [7]:
import logging
def config_notebooks_logger():
    logging.config.fileConfig('logging.conf')
    msglogger = logging.getLogger()
    msglogger.info('Logging configured successfully')
    return msglogger

In [8]:
import argparse
import distiller

msglogger = config_notebooks_logger()

parser = argparse.ArgumentParser()
distiller.quantization.add_post_train_quant_args(parser)
args = parser.parse_args(args= [])
# args.qe_config_file = '/home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet18_imagenet_post_train.yaml'

Logging configured successfully


<a id='stats'> </a>

# Correct way of getting statistics

In [ ]:
cpu_model = distiller.make_non_parallel_copy(model)

In [ ]:
from distiller.data_loggers import collect_quant_stats, QuantCalibrationStatsCollector, collector_context


args.qe_calibration = 0.2
if args.qe_calibration:
    
    cpu_model = distiller.make_non_parallel_copy(model).cpu()
    
    distiller.utils.assign_layer_fq_names(cpu_model)
    msglogger.info("Generating quantization calibration stats based on {0} users".format(args.qe_calibration))
    collector = distiller.data_loggers.QuantCalibrationStatsCollector(cpu_model)
    with collector_context(collector):
        eval_model(train_loader_gpu,cpu_model,'cuda',print_freq=30)
        # Here call your model evaluation function, making sure to execute only
        # the portion of the dataset specified by the qe_calibration argument
    yaml_path = './act_quantization_stats.yaml'
    collector.save(yaml_path)

<a href='#base'> </a>

# Base precision

In [9]:
if torch.cuda.is_available():
    %time eval_model(dataloader,model,'cuda', print_freq=40)

7000 samples (40 per mini-batch)
[ 40/175] Top1: 76.250  Top5: 93.688  Loss: 0.885
[ 80/175] Top1: 78.156  Top5: 93.719  Loss: 0.845
[120/175] Top1: 78.208  Top5: 93.896  Loss: 0.838
[160/175] Top1: 78.328  Top5: 93.922  Loss: 0.833
----------
Overall ==> Top1: 78.286  Top5: 93.943  Loss: 0.832  PPL: 2.298
CPU times: user 1min 29s, sys: 841 ms, total: 1min 29s
Wall time: 1min 31s


In [10]:
args.quantize_eval = True

In [11]:
from copy import deepcopy
def eval_quantized(model, args):
    if args.quantize_eval:
        quantizer = distiller.quantization.PostTrainLinearQuantizer.from_args(deepcopy(model), args)
        # dummy = distiller.get_dummy_input(model.input_shape)
        dummy = distiller.get_dummy_input(input_shape=model.input_shape)
        quantizer.prepare_model(dummy)
        eval_model(dataloader, quantizer.model, 'cuda', print_freq=30)

<a id='8bit'> </a>

# 8 bit quantization

In [12]:
args.qe_config_file = '/home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet/resnet18/resnet18_imagenet_post_train.yaml'
eval_quantized(model, args)

Reading configuration from: /home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet/resnet18/resnet18_imagenet_post_train.yaml
Found component of class PostTrainLinearQuantizer: Name: post_train_quantizer ; Section: quantizers
Loading activation stats from: /home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet/acts_quantization_stats.yaml
Preparing model for quantization using PostTrainLinearQuantizer
Applying batch-norm folding ahead of post-training quantization
Propagating output statistics from BN modules to folded modules
Optimizing output statistics for modules followed by ReLU/Tanh/Sigmoid
Updated stats saved to ./quant_stats_after_prepare_model.yaml
Per-layer quantization parameters saved to ./layer_quant_params.yaml


7000 samples (40 per mini-batch)
[ 30/175] Top1: 78.583  Top5: 93.750  Loss: 0.813
[ 60/175] Top1: 78.500  Top5: 94.083  Loss: 0.822
[ 90/175] Top1: 78.389  Top5: 93.944  Loss: 0.832
[120/175] Top1: 77.875  Top5: 93.854  Loss: 0.842
[150/175] Top1: 78.250  Top5: 93.850  Loss: 0.834
----------
Overall ==> Top1: 78.214  Top5: 93.886  Loss: 0.837  PPL: 2.310


<a id='4bit'> </a>

# 4 bit quantization

## Run 1
```python
class: PostTrainLinearQuantizer
bits_activations: 4
bits_parameters: 4
bits_accum: 16
mode: ASYMMETRIC_UNSIGNED
per_channel_wts: True
clip_acts: AVG

```

In [13]:
args.qe_config_file = '/home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet/resnet18/resnet18_imagenet_post_train_4bit.yaml'
eval_quantized(model, args)

Reading configuration from: /home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet/resnet18/resnet18_imagenet_post_train_4bit.yaml
Found component of class PostTrainLinearQuantizer: Name: post_train_quantizer ; Section: quantizers
Loading activation stats from: /home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet/acts_quantization_stats.yaml
Preparing model for quantization using PostTrainLinearQuantizer
Applying batch-norm folding ahead of post-training quantization
Propagating output statistics from BN modules to folded modules
Optimizing output statistics for modules followed by ReLU/Tanh/Sigmoid
Updated stats saved to ./quant_stats_after_prepare_model.yaml
Per-layer quantization parameters saved to ./layer_quant_params.yaml


7000 samples (40 per mini-batch)
[ 30/175] Top1: 28.167  Top5: 51.000  Loss: 3.792
[ 60/175] Top1: 28.000  Top5: 50.792  Loss: 3.782
[ 90/175] Top1: 26.306  Top5: 50.389  Loss: 3.864
[120/175] Top1: 26.812  Top5: 51.083  Loss: 3.829
[150/175] Top1: 26.833  Top5: 50.600  Loss: 3.832
----------
Overall ==> Top1: 27.157  Top5: 50.829  Loss: 3.805  PPL: 44.947


## Uprava parametrov
## Run 2
/home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet18_imagenet_post_train_4bit2.yaml

```python
quantizers:
  post_train_quantizer:
    class: PostTrainLinearQuantizer
    bits_activations: 4
    bits_parameters: 4
    bits_accum: 16

    mode: ASYMMETRIC_UNSIGNED
    
    model_activation_stats: /home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/acts_quantization_stats.yaml
    per_channel_wts: True
    clip_acts: AVG

    overrides:
      fc:
        clip_acts: NONE  # Don't clip activations in last layer before softmax
```

In [14]:
args.qe_config_file = '/home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet/resnet18/resnet18_imagenet_post_train_4bit2.yaml'
eval_quantized(model, args)

Reading configuration from: /home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet/resnet18/resnet18_imagenet_post_train_4bit2.yaml
Found component of class PostTrainLinearQuantizer: Name: post_train_quantizer ; Section: quantizers
Loading activation stats from: /home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet/acts_quantization_stats.yaml
Preparing model for quantization using PostTrainLinearQuantizer
Applying batch-norm folding ahead of post-training quantization
Propagating output statistics from BN modules to folded modules
Optimizing output statistics for modules followed by ReLU/Tanh/Sigmoid
Updated stats saved to ./quant_stats_after_prepare_model.yaml
Per-layer quantization parameters saved to ./layer_quant_params.yaml


7000 samples (40 per mini-batch)
[ 30/175] Top1: 21.500  Top5: 43.167  Loss: 4.409
[ 60/175] Top1: 22.917  Top5: 44.375  Loss: 4.271
[ 90/175] Top1: 22.917  Top5: 44.444  Loss: 4.244
[120/175] Top1: 22.896  Top5: 43.937  Loss: 4.247
[150/175] Top1: 22.717  Top5: 44.133  Loss: 4.248
----------
Overall ==> Top1: 22.786  Top5: 44.029  Loss: 4.263  PPL: 71.023


```python
 overrides:
    # First and last layers in 8-bits
      conv1:
        bits_weights: 8
        bits_activations: 8
      fc:
        bits_weights: 8
        bits_activations: 8
        clip_acts: NONE  # Don't clip activations in last layer before softmax
```

In [15]:
args.qe_config_file = '/home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet/resnet18/resnet18_imagenet_post_train_4bit3.yaml'
eval_quantized(model, args)

Reading configuration from: /home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet/resnet18/resnet18_imagenet_post_train_4bit3.yaml
Found component of class PostTrainLinearQuantizer: Name: post_train_quantizer ; Section: quantizers
Loading activation stats from: /home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet/acts_quantization_stats.yaml
Preparing model for quantization using PostTrainLinearQuantizer
Applying batch-norm folding ahead of post-training quantization
Propagating output statistics from BN modules to folded modules
Optimizing output statistics for modules followed by ReLU/Tanh/Sigmoid
Updated stats saved to ./quant_stats_after_prepare_model.yaml
Per-layer quantization parameters saved to ./layer_quant_params.yaml


7000 samples (40 per mini-batch)
[ 30/175] Top1: 0.167  Top5: 0.833  Loss: 9.153
[ 60/175] Top1: 0.208  Top5: 1.167  Loss: 9.105
[ 90/175] Top1: 0.139  Top5: 1.000  Loss: 9.076
[120/175] Top1: 0.146  Top5: 1.021  Loss: 9.098
[150/175] Top1: 0.133  Top5: 0.900  Loss: 9.102
----------
Overall ==> Top1: 0.143  Top5: 0.871  Loss: 9.095  PPL: 8908.387


## Mixed model

```python
quantizers:
  post_train_quantizer:
    class: PostTrainLinearQuantizer
    bits_activations: 8
    bits_parameters: 4
    bits_accum: 16

    mode: ASYMMETRIC_UNSIGNED
   
    model_activation_stats: /home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/acts_quantization_stats.yaml
    per_channel_wts: True
    clip_acts: AVG

    overrides:
    # First and last layers in 8-bits
      conv1:
        bits_weights: 8
        bits_activations: 8
      fc:
        bits_weights: 8
        bits_activations: 8
        clip_acts: NONE  # Don't clip activations in last layer before softmax
```

In [16]:
args.qe_config_file = '/home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet/resnet18/resnet18_imagenet_post_train_4bit4.yaml'
eval_quantized(model, args)

Reading configuration from: /home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet/resnet18/resnet18_imagenet_post_train_4bit4.yaml
Found component of class PostTrainLinearQuantizer: Name: post_train_quantizer ; Section: quantizers
Loading activation stats from: /home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet/acts_quantization_stats.yaml
Preparing model for quantization using PostTrainLinearQuantizer
Applying batch-norm folding ahead of post-training quantization
Propagating output statistics from BN modules to folded modules
Optimizing output statistics for modules followed by ReLU/Tanh/Sigmoid
Updated stats saved to ./quant_stats_after_prepare_model.yaml
Per-layer quantization parameters saved to ./layer_quant_params.yaml


7000 samples (40 per mini-batch)
[ 30/175] Top1: 0.000  Top5: 1.000  Loss: 9.346
[ 60/175] Top1: 0.083  Top5: 1.125  Loss: 9.283
[ 90/175] Top1: 0.083  Top5: 1.028  Loss: 9.266
[120/175] Top1: 0.125  Top5: 1.104  Loss: 9.300
[150/175] Top1: 0.133  Top5: 1.117  Loss: 9.333
----------
Overall ==> Top1: 0.157  Top5: 1.171  Loss: 9.323  PPL: 11187.213


<a id='8bit-loss'> </a>

# 8 bit Loss-Aware quantization

Distiller supports Loss-Aware quantization in its sample application.

In [1]:
# ! python /home/bohumil/FIIT/BP/BP/Zdroje_kod/distiller/examples/classifier_compression/compress_classifier.py --arch resnet18 --pretrained --quantize-eval --qe-config-file /home/bohumil/FIIT/BP/BP/Zdroje_kod/distiller/examples/quantization/post_train_quant/resnet18_imagenet_post_train_lapq.yaml --validation-split 0.3  -j 2 -b 20 --eval /home/bohumil/FIIT/BP/BP/Zdroje_kod/imagenet/train
! python /home/bohumil/FIIT/BP/BP/Zdroje_kod/distiller/examples/classifier_compression/compress_classifier.py --arch resnet18 --print-freq 30 -b 20 -j 1 --pretrained --validation-split 0.2 --eval --quantize-eval --qe-config-file /home/bohumil/FIIT/BP/BP/Zdroje_kod/distiller/examples/quantization/post_train_quant/resnet18_imagenet_post_train_lapq.yaml --qe-lapq --lapq-search-clipping /home/bohumil/FIIT/BP/BP/Zdroje_kod/imagenet/

Log file for this run: /home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet/logs/2020.04.23-135409/2020.04.23-135409.log
Random seed: 0

--------------------------------------------------------
Logging to TensorBoard - remember to execute the server:
> tensorboard --logdir='./logs'

=> created a pretrained resnet18 model with the imagenet dataset
Dataset sizes:
	test=19439
Dataset sizes:
	test=19439
Reading configuration from: /home/bohumil/FIIT/BP/BP/Zdroje_kod/distiller/examples/quantization/post_train_quant/resnet18_imagenet_post_train_lapq.yaml
Found component of class PostTrainLinearQuantizer: Name: post_train_quantizer ; Section: quantizers
Loading activation stats from: /home/bohumil/FIIT/BP/BP/Zdroje_kod/distiller/examples/quantization/post_train_quant/stats/resnet18_quant_stats.yaml
Initializing quantizer...
Preparing model for quantization using PostTrainLinearQuantizer
Applying batch-norm folding ahead of post-training quantization
Propagating output statistics f

In [7]:
! python /home/bohumil/FIIT/BP/BP/Zdroje_kod/distiller/examples/classifier_compression/compress_classifier.py --eval --qe --qe-lapq -a resnet18 --pretrained /home/bohumil/FIIT/BP/BP/Zdroje_kod/imagenet/ --lapq-eval-size 0.01 --lapq-maxiter 2 --qe-config-file /home/bohumil/FIIT/BP/BP/Zdroje_kod/distiller/examples/quantization/post_train_quant/resnet18_imagenet_post_train_lapq.yaml -b 50 -j 2 --lapq-init-mode L3 --lapq-init-method powell --lapq-eval-memoize-dataloader --det --lapq-search-clipping

Log file for this run: /home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet/logs/2020.04.23-150406/2020.04.23-150406.log
Random seed: 0

--------------------------------------------------------
Logging to TensorBoard - remember to execute the server:
> tensorboard --logdir='./logs'

=> created a pretrained resnet18 model with the imagenet dataset
Dataset sizes:
	test=194
Dataset sizes:
	test=19439
Reading configuration from: /home/bohumil/FIIT/BP/BP/Zdroje_kod/distiller/examples/quantization/post_train_quant/resnet18_imagenet_post_train_lapq.yaml
Found component of class PostTrainLinearQuantizer: Name: post_train_quantizer ; Section: quantizers
Loading activation stats from: /home/bohumil/FIIT/BP/BP/Zdroje_kod/distiller/examples/quantization/post_train_quant/stats/resnet18_quant_stats.yaml
Initializing quantizer...
Preparing model for quantization using PostTrainLinearQuantizer
Applying batch-norm folding ahead of post-training quantization
Propagating output statistics fro

In [ ]:
# args.qe_config_file = '/home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet18_imagenet_post_train.yaml'
args.qe_config_file = '/home/bohumil/FIIT/BP/BP/Zdroje_kod/distiller/examples/quantization/post_train_quant/resnet18_imagenet_post_train_lapq.yaml'
# --qe-config-file /home/bohumil/FIIT/BP/BP/Zdroje_kod/distiller/examples/quantization/post_train_quant/resnet18_imagenet_post_train_lapq.yaml --qe-lapq --lapq-search-clipping
args.qe_lapq = true
args.laps_search_clipping = true
args.lapq_maxiter = 2
args.lapq_init_mode = 'L1'
args.lapq_init_method = 'Powell'
args.lapq_eval_size = 0.1
args.lapq_eval_memoize_dataloader = true
eval_quantized(model, args)

<a id='4bit-loss'> </a>

# 4 bit Loss-Aware quantization